In [45]:
#Se importan los paquetes necesarios para los cálculos.
import pandas as pd #manejo de datos
import numpy as np #parquete de matemática
import scipy.stats as sp #paquete de estadísticas
from sqlalchemy import create_engine #consultas a sql
import sqlalchemy #manejo de sql
import sys #manejo del sistema para recibir argumentos
import warnings #
from sqlalchemy import text
warnings.filterwarnings("ignore")
import pyodbc

import funciones

In [46]:
#Conexión a la base de datos
server = '10.0.0.153'
user = "testipc2"
pas = "Abc$2020"
#user = input('Ingrese el usuario: ')
#pas = input('Ingrese la contraseña: ')
try:
    #driver = 'SQL Server Native Client 11.0'
    driver = 'ODBC Driver 17 for SQL Server'
    database_connection = f'mssql://{user}:{pas}@{server}/master?driver={driver}'
    engine = create_engine(database_connection)
    connection = engine.connect()
    print('Conexión exitosa')
except:
    print('Fallo en la conexión')

Conexión exitosa


In [47]:
anio = int(input('Ingrese el año: '))
mes = int(input('Ingrese el mes: '))
'''anio = 2024
mes = 1'''

'anio = 2024\nmes = 1'

In [48]:
#aqui se empieza la parte de la conexion con el otro server
#Almacenamos la información del servidor para poder realizar la conexion más adelante
server = 'ipcprod.database.windows.net'
user = 'ipcreader'
password = '1pc/*2023'
database = 'db-indices'

In [49]:
#Se crea un string para realizar la conexión, con la informacion del servidor almacenada previamente
conn_str = (
    f'Driver={{ODBC Driver 17 for SQL Server}};'
    f'Server={server};'
    f'Database={database};'
    f'Uid={user};'
    f'Pwd={password};'
)

In [50]:
try:
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    
    sql_command = f"EXEC [dbo].[sp_get_precios_recolectados_mes] {anio}, {mes}"
    cursor.execute(sql_command)
    boletas01 = pd.DataFrame([tuple(row) for row in cursor.fetchall()], columns=[column[0] for column in cursor.description])

    conn.close()
    print("Connection successful")
except Exception as e:
    print(f"Error: {e}")

Connection successful


In [51]:
boletas02 = boletas01
boletas02['codigo_articulo'] = boletas01['codigo_articulo'].replace(funciones.recodificacion) #esta parte hace la recodificacion con el diccionario de los codigos en el archivo funciones
boletas02['decada'] = boletas02['decada'].replace(funciones.decadas)
boletas02['nt_tipo'] = boletas02['nt_tipo'].replace({351761:'S', 351762:'S'})
boletas02['nt_tipo'] = boletas02['nt_tipo'].fillna('N')
boletas02['cantidad_anterior'] = pd.to_numeric(boletas02['cantidad_anterior'], errors='coerce')
boletas02['cantidad_actual'] = pd.to_numeric(boletas02['cantidad_actual'], errors='coerce')
boletas02['precio_anterior'] = pd.to_numeric(boletas02['precio_anterior'], errors='coerce')
boletas02['precio_actual'] = pd.to_numeric(boletas02['precio_actual'], errors='coerce')
boletas02['decada'] = pd.to_numeric(boletas02['decada'], errors='coerce')
boletas02['region'] = boletas02['region'].astype(str)
boletas02['codigo_articulo'] = boletas02['codigo_articulo'].astype(str)
boletas02 = boletas02[['region', 'codigo_articulo', 'ine_poll_id', 'cantidad_anterior', 'cantidad_actual', 'precio_anterior', 'precio_actual', 'nt_tipo', 'decada']]
boletas02.rename(columns={'region':'RegCod', 'codigo_articulo':'ArtCod', 'ine_poll_id':'BolNum', 'cantidad_anterior':'UraChi', 'cantidad_actual':'UreCan', 'precio_anterior':'ArtPhi', 'precio_actual':'ArtPac', 'nt_tipo':'ArtCR', 'decada':'PerSem'}, inplace=True)
boletas02['RegCod'] = boletas02['RegCod'].replace({'1':'01', '2':'02', '3':'03', '4':'04', '5':'05', '6':'06', '7':'07','8':'08',})

boletas02

,RegCod,ArtCod,BolNum,UraChi,UreCan,ArtPhi,ArtPac,ArtCR,PerSem
0,01,053110202,34-4-2024,1.0,1.0,145.0,145.00,N,1
1,01,031220302,36-4-2024,1.0,1.0,250.0,250.00,N,1
2,01,072110201,42-4-2024,1.0,1.0,400.0,375.00,N,1
3,01,041110103,43-4-2024,1.0,1.0,1700.0,1700.00,N,1
4,01,3211051,52-4-2024,0.0,1.0,0.0,279.99,N,1
...,...,...,...,...,...,...,...,...,...
65126,08,111110101,23430-4-2024,0.0,1.0,0.0,35.00,N,3
65127,08,111110201,23430-4-2024,0.0,1.0,0.0,35.00,N,3
65128,08,111110201,23430-4-2024,0.0,1.0,0.0,35.00,N,3
65129,08,111110201,23430-4-2024,0.0,1.0,0.0,35.00,N,3


In [52]:
#crea un dataframe con los articulos recodificados a la base 2010
similar = boletas02[boletas02['ArtCod'].str.len() == 9]
similar

,RegCod,ArtCod,BolNum,UraChi,UreCan,ArtPhi,ArtPac,ArtCR,PerSem
0,01,053110202,34-4-2024,1.0,1.0,145.0,145.0,N,1
1,01,031220302,36-4-2024,1.0,1.0,250.0,250.0,N,1
2,01,072110201,42-4-2024,1.0,1.0,400.0,375.0,N,1
3,01,041110103,43-4-2024,1.0,1.0,1700.0,1700.0,N,1
5,01,072110302,54-4-2024,946.0,946.0,70.0,70.0,N,1
...,...,...,...,...,...,...,...,...,...
65126,08,111110101,23430-4-2024,0.0,1.0,0.0,35.0,N,3
65127,08,111110201,23430-4-2024,0.0,1.0,0.0,35.0,N,3
65128,08,111110201,23430-4-2024,0.0,1.0,0.0,35.0,N,3
65129,08,111110201,23430-4-2024,0.0,1.0,0.0,35.0,N,3


In [53]:
#se importan los precios del mes de diciembre de 2023 que son los ultimos precios que se conocen de la base 2010
boldic2023 = pd.read_excel("boletadic2023.xlsx")
boldic2023['ArtCod'] = boldic2023['ArtCod'].astype(str)
boldic2023.loc[boldic2023['ArtCod'].str.len() == 8, 'ArtCod'] = '0' + boldic2023['ArtCod']
boldic2023['RegCod'] = boldic2023['RegCod'].astype(str)
boldic2023['RegCod'] = boldic2023['RegCod'].replace({'1':'01', '2':'02', '3':'03', '4':'04', '5':'05', '6':'06', '7':'07','8':'08',})

In [54]:
#se lee el archivo que tienen los precios imputados del mes a evaluar
imputname01 = 'paraimputar'
imputname02 = '.xlsx'
inputname = f"{imputname01}_{mes}_{anio}{imputname02}"

para_imputar = pd.read_excel(inputname)

In [55]:
#se obtienen los productos que son "similares" a los productos imputados
#se hacen los ajustes necesarios al dataframe que contiene la lista de productos imputados
para_imputar['ArtCod'] = para_imputar['ArtCod'].astype(str)
para_imputar.loc[para_imputar['ArtCod'].str.len() == 8, 'ArtCod'] = '0' + para_imputar['ArtCod']

#se genera un dataframe que corta el codigo del articulo para poder evaluarlos a nivel de subgrupo
ref_similar = pd.DataFrame(para_imputar['ArtCod'].str[:-2])

#ref_similar01 = pd.DataFrame(para_imputar['ArtCod'].str[:-1])
#ref_similar.loc[ref_similar['ArtCod'].str.len() == 6, 'ArtCod'] = '0' + ref_similar['ArtCod']

In [56]:
#aqui voy a usar los datos de referencia similares para sacar de la base de datos del mes los articulos de referencia
ref_sim = ref_similar['ArtCod']
#ref_sim01 = ref_similar01['ArtCod']
ref_sim02 = para_imputar['ArtCod'] #este sirve para tener una serie de referencia con el string de codigo de articulo completo

#se obtienen los articulos que se encuentran dentro de un grupo similar para la base de datos del mes actual
similar01 = similar[similar['ArtCod'].str.startswith(tuple(ref_sim))]
similar01

,RegCod,ArtCod,BolNum,UraChi,UreCan,ArtPhi,ArtPac,ArtCR,PerSem
17,01,011130102,60-4-2024,1.0,1.0,5.20,5.20,N,1
29,01,011230101,60-4-2024,454.0,454.0,14.30,12.15,N,1
39,01,011430103,60-4-2024,450.0,450.0,23.65,23.65,N,1
40,01,011430101,60-4-2024,450.0,450.0,36.85,38.85,N,1
44,01,011520101,60-4-2024,443.0,400.0,11.60,11.60,S,1
...,...,...,...,...,...,...,...,...,...
65060,08,011230101,22115-4-2024,0.0,454.0,0.00,15.00,N,3
65061,08,011430105,22115-4-2024,0.0,454.0,0.00,30.00,N,3
65073,08,061110501,22123-4-2024,0.0,120.0,0.00,10.00,N,3
65110,08,121210301,22575-4-2024,1.0,1.0,6.00,6.00,N,3


In [57]:
#se calcula la variacion para los articulos similares del mes actual
similar02 = similar01
similar02["variacion"] = ((similar02['ArtPac']*similar02['UraChi'])/(similar02['ArtPhi']*similar02['UreCan']))
similar02["variacion"] = similar02["variacion"].replace ([0, np.NaN,np.inf ], 1)

similar02

,RegCod,ArtCod,BolNum,UraChi,UreCan,ArtPhi,ArtPac,ArtCR,PerSem,variacion
17,01,011130102,60-4-2024,1.0,1.0,5.20,5.20,N,1,1.000000
29,01,011230101,60-4-2024,454.0,454.0,14.30,12.15,N,1,0.849650
39,01,011430103,60-4-2024,450.0,450.0,23.65,23.65,N,1,1.000000
40,01,011430101,60-4-2024,450.0,450.0,36.85,38.85,N,1,1.054274
44,01,011520101,60-4-2024,443.0,400.0,11.60,11.60,S,1,1.107500
...,...,...,...,...,...,...,...,...,...,...
65060,08,011230101,22115-4-2024,0.0,454.0,0.00,15.00,N,3,1.000000
65061,08,011430105,22115-4-2024,0.0,454.0,0.00,30.00,N,3,1.000000
65073,08,061110501,22123-4-2024,0.0,120.0,0.00,10.00,N,3,1.000000
65110,08,121210301,22575-4-2024,1.0,1.0,6.00,6.00,N,3,1.000000


In [58]:
#se obtienen un dataframe que contiene solamente el codigo del articulo y la variacion
variacion = similar02
variacion = similar02[['ArtCod', 'variacion']]
variacion

,ArtCod,variacion
17,011130102,1.000000
29,011230101,0.849650
39,011430103,1.000000
40,011430101,1.054274
44,011520101,1.107500
...,...,...
65060,011230101,1.000000
65061,011430105,1.000000
65073,061110501,1.000000
65110,121210301,1.000000


In [59]:
#se calcula la variacion acumulada para los grupos de los productos que son similares a los imputados
var_ac = pd.DataFrame(variacion.groupby(ref_sim02)['variacion'].agg(sp.gmean)).reset_index()
#variacion[variacion['ArtCod'].str.startswith(tuple(ref_sim01))]
var_ac

,ArtCod,variacion
0,011920101,1.000000
1,031230601,0.849650
2,051120101,1.000000
3,051120201,1.054274
4,054120401,1.107500
5,061110502,1.000000
6,083120101,1.043997
7,092110201,1.000000
8,093210201,0.745856
9,093310101,1.000000


In [60]:
#se obtienen los precios del mes anterior para los articulos a imputar
#como esto es diciembre el precio a usar es el precio actual de diciembre
precioant = boldic2023[boldic2023['ArtCod'].str.startswith(tuple(ref_sim))]
funcionagg = {'ArtPac': 'median', 'ArtPhi': 'median', 'UraChi': 'median', 'UreCan': 'median', 'RegCod': 'first', 'BolNum': 'first', 'ArtCR': 'first', 'PerSem': 'first'}
pant_median = precioant.groupby('ArtCod').agg(funcionagg).reset_index() #utilice la mediana de los precios para no tener mucho problema con valores extremos
p_ant_imput = pd.merge(pant_median, para_imputar, on='ArtCod', how='inner') #se obtienen un dataframe con los precios a imputar
p_ant_imput

,ArtCod,ArtPac,ArtPhi,UraChi,UreCan,RegCod,BolNum,ArtCR,PerSem
0,011110102,5.00,5.00,460.0,460.0,01,313510,N,1
1,011130104,3.60,3.60,161.0,166.0,01,313550,N,1
2,011130301,10.00,10.00,1.0,1.0,01,313519,N,1
3,011230102,17.00,17.00,460.0,460.0,01,313506,N,1
4,011230103,15.00,14.50,460.0,460.0,01,313506,N,1
...,...,...,...,...,...,...,...,...,...
77,122210101,156.50,139.50,1.0,1.0,01,313518,N,1
78,122210201,350.00,345.00,1.0,1.0,01,313778,N,1
79,122210301,59.00,60.00,1.0,1.0,01,313518,N,1
80,123110103,182.27,182.27,1.0,1.0,01,313793,N,1


In [61]:
#se obtiene un dataframe con los precios a imputar y la variacion acumulada anteriormente
#como no siempre estan todos los que se van a imputar en la boleta de diciembre para los precios anteriores entonces voy a llenar con 1
p_imput = pd.merge(p_ant_imput, var_ac, on='ArtCod', how='left')
p_imput['variacion'].fillna(1, inplace=True)
p_imput['ArtPimpt'] = (p_imput['ArtPac']*p_imput['variacion']) #se calcula el precio imputado

In [62]:
pimput = p_imput
pimput['ArtPac'] = pimput['ArtPimpt']
pimput.drop(columns=['ArtPimpt'], inplace=True)
pimput

,ArtCod,ArtPac,ArtPhi,UraChi,UreCan,RegCod,BolNum,ArtCR,PerSem,variacion
0,011110102,5.00,5.00,460.0,460.0,01,313510,N,1,1.0
1,011130104,3.60,3.60,161.0,166.0,01,313550,N,1,1.0
2,011130301,10.00,10.00,1.0,1.0,01,313519,N,1,1.0
3,011230102,17.00,17.00,460.0,460.0,01,313506,N,1,1.0
4,011230103,15.00,14.50,460.0,460.0,01,313506,N,1,1.0
...,...,...,...,...,...,...,...,...,...,...
77,122210101,156.50,139.50,1.0,1.0,01,313518,N,1,1.0
78,122210201,350.00,345.00,1.0,1.0,01,313778,N,1,1.0
79,122210301,59.00,60.00,1.0,1.0,01,313518,N,1,1.0
80,123110103,182.27,182.27,1.0,1.0,01,313793,N,1,1.0


In [63]:
duplicated_rows = []
for i in range(1, 9):
    duplicated_rows.append(pimput.assign(RegCod=str(i).zfill(2)))

p_imp = pd.concat(duplicated_rows, ignore_index=True)
p_imp.drop(columns=['variacion'], inplace=True)
desired_order = ['RegCod', 'ArtCod', 'BolNum', 'UraChi', 'UreCan', 'ArtPhi', 'ArtPac', 'ArtCR', 'PerSem']
p_imp = p_imp[desired_order]
p_imp

,RegCod,ArtCod,BolNum,UraChi,UreCan,ArtPhi,ArtPac,ArtCR,PerSem
0,01,011110102,313510,460.0,460.0,5.00,5.00,N,1
1,01,011130104,313550,161.0,166.0,3.60,3.60,N,1
2,01,011130301,313519,1.0,1.0,10.00,10.00,N,1
3,01,011230102,313506,460.0,460.0,17.00,17.00,N,1
4,01,011230103,313506,460.0,460.0,14.50,15.00,N,1
...,...,...,...,...,...,...,...,...,...
651,08,122210101,313518,1.0,1.0,139.50,156.50,N,1
652,08,122210201,313778,1.0,1.0,345.00,350.00,N,1
653,08,122210301,313518,1.0,1.0,60.00,59.00,N,1
654,08,123110103,313793,1.0,1.0,182.27,182.27,N,1


In [64]:
p_imp.to_excel("imputados.xlsx")

Esta parte no estoy segura si voy bien -------------------------------------------------------------------

In [65]:
test01 = pd.DataFrame(precioant.groupby('ArtCod')['ArtPac'].mean()).reset_index()
test01

,ArtCod,ArtPac
0,011110101,6.610336
1,011110102,4.871467
2,011110103,7.209869
3,011130101,3.672939
4,011130102,3.949061
...,...,...
139,123110101,5698.510000
140,123110102,1650.636667
141,123110103,3275.542500
142,123110104,300.000000


In [66]:
# Assuming your DataFrame is named df
# Winsorizing 'ArtPac' to handle extreme values
winsorized_artpac = sp.mstats.winsorize(precioant['ArtPac'], limits=[0.025, 0.025])  # Adjust limits as needed
# Creating a new DataFrame with 'ArtCod' and the winsorized 'ArtPac'
df_winsorized = pd.DataFrame({'ArtCod': precioant['ArtCod'], 'Winsorized_ArtPac': winsorized_artpac})
# Grouping by 'ArtCod' and calculating the mean of winsorized 'ArtPac'
result = df_winsorized.groupby('ArtCod')['Winsorized_ArtPac'].mean().reset_index()
result

,ArtCod,Winsorized_ArtPac
0,011110101,6.610336
1,011110102,4.871467
2,011110103,7.209869
3,011130101,3.672939
4,011130102,3.949061
...,...,...
139,123110101,472.000000
140,123110102,472.000000
141,123110103,209.542500
142,123110104,300.000000
